In [1]:
!pip install fastai --upgrade > /dev/null
!pip install requests

In [1]:
from fastai.vision.all import *

In [2]:
p = Path('.')
p.ls()

(#80) [Path('.ipynb_checkpoints'),Path('01_Concepts-Data_loading.svg'),Path('02_Concepts-Model_training.svg'),Path('03_Concepts-Learner_lifecycle.svg'),Path('04_Learner-Factory_methods_to_create_an_instance.svg'),Path('05_Learner-Init___Attributes.svg'),Path('06_Learner-Training_methods.svg'),Path('07_Learner-Inference_methods.svg'),Path('08_Diagnostics-How_to_debug.svg'),Path('09_Show-Inputs_targets_and_predictions.svg')...]

In [3]:
files = get_files(p,extensions=".SVG")
files

(#75) [Path('01_Concepts-Data_loading.svg'),Path('02_Concepts-Model_training.svg'),Path('03_Concepts-Learner_lifecycle.svg'),Path('04_Learner-Factory_methods_to_create_an_instance.svg'),Path('05_Learner-Init___Attributes.svg'),Path('06_Learner-Training_methods.svg'),Path('07_Learner-Inference_methods.svg'),Path('08_Diagnostics-How_to_debug.svg'),Path('09_Show-Inputs_targets_and_predictions.svg'),Path('10_Show-Images.svg')...]

In [4]:
titles = pd.read_csv("titles.csv", sep=';', header=None)
titles

,0,1,2
0,1,Concepts,Concepts - Data loading
1,2,Concepts,Concepts - Model training
2,3,Concepts,Concepts - Learner lifecycle
3,4,Learner,Learner - Factory methods to create an instance
4,5,Learner,Learner - Init & Attributes
...,...,...,...
70,71,Loss,Loss functions - Classification & Regression
71,72,Summary,Summary - DataBlock
72,73,Summary,Summary - DataLoaders
73,74,Summary,Summary - Learner


In [5]:
#numfiles = L(zip(files.map(lambda n: int(n.name[11-len(n.name):-4])),files))
numfiles = L(zip(files.map(lambda n: int(n.name[:2])),files))

In [6]:
targets = L()
for num,file in numfiles:
    target = ('0' if num<=9 else '')+str(num)+'_'+titles[2].iloc[num-1].replace(' - ','-').replace(' ','_').replace(',','').replace('/','_').replace('&','_').replace(':','_')+'.svg'
    targets.append((num,target))
    file.rename(target)
targets.sort(key=lambda t:t[0])

titles[3] = targets.itemgot(1)
titles

,0,1,2,3
0,1,Concepts,Concepts - Data loading,01_Concepts-Data_loading.svg
1,2,Concepts,Concepts - Model training,02_Concepts-Model_training.svg
2,3,Concepts,Concepts - Learner lifecycle,03_Concepts-Learner_lifecycle.svg
3,4,Learner,Learner - Factory methods to create an instance,04_Learner-Factory_methods_to_create_an_instance.svg
4,5,Learner,Learner - Init & Attributes,05_Learner-Init___Attributes.svg
...,...,...,...,...
70,71,Loss,Loss functions - Classification & Regression,71_Loss_functions-Classification___Regression.svg
71,72,Summary,Summary - DataBlock,72_Summary-DataBlock.svg
72,73,Summary,Summary - DataLoaders,73_Summary-DataLoaders.svg
73,74,Summary,Summary - Learner,74_Summary-Learner.svg


In [12]:
menuhtml = "<h1>fastai v2</h1>\n"
for title1 in titles[1].unique():
    menuhtml += f"<h2>{title1}</h2>\n"
    menuhtml += "<ul>"
    for title2 in titles[2][titles[1]==title1]:
        num = titles[3][(titles[1]==title1) & (titles[2]==title2)].item()[:2]
        menuhtml += f"<li><a href=\"#{num}\">{title2}</a></li>\n"
    menuhtml += "</ul>"

In [13]:
mainhtml = ""
for title1 in titles[1].unique():
    mainhtml += f"<h2>{title1}</h2>\n"
    for title2 in titles[2][titles[1]==title1]:
        num = titles[3][(titles[1]==title1) & (titles[2]==title2)].item()[:2]
        mainhtml += f"<p><a href=\"https://docs.fast.ai\" target=\"_blank\"><img id=\"{num}\" src=\"/images/fastaidocs/{titles[3][(titles[1]==title1) & (titles[2]==title2)].item()}\"/></p>\n"

In [14]:
main1html = """---
permalink: /fastaidocs/
layout: null
search_exclude: true
---

<html>
<head>
<style>
/* The sidenav */
.sidenav {
  padding:10px;
  height: 100%;
  width: 280px;
  position: fixed;
  z-index: 1;
  top: 0;
  left: 0;
  color: #7F7F7F;
  background-color: #F7F7F7;
  overflow-x: hidden;
  border-right: solid 1px;
}

ul {
  padding-left: 10px;
}

li {
  list-style-type: none;
  padding-left: 0px;
  padding-bottom: 5px;
}

a {
  outline: none;
  text-decoration: none;
  color: #7F7F7F;
}

a:hover {
  border-bottom: 1px solid;
}

/* Page content */
.main {
  margin-left: 300px; /* Same as the width of the sidenav */
  padding: 0px 10px;
}

.main h2 {
  color: #7F7F7F;
  background-color: #F7F7F7;
  font-size: 50px;
  border-top: 1px solid;
  border-bottom: 1px solid;
  padding: 10px;
}

.main img {
  width: 100%;
  border: 1px solid #E7E7E7;
}

</style>
</head>

<body>

<div class="sidenav">

"""

In [15]:
main2html = """
</div>

<div class="main">
"""

In [16]:
main3html = """
</div>

</body>

</html>"""

In [17]:
text_file = open("index.html", "w")
text_file.write(main1html + menuhtml + main2html + mainhtml + main3html)
text_file.close()

In [50]:
# TEST : can we get doc urls from slide title ?
# no - doesn't work

import urllib.parse
import requests
import re

for title2 in titles[2]:
    print(title2)
    encodedsearch = urllib.parse.quote(title2)
    print(encodedsearch)
    searchurl = f"https://cse.google.com/cse/element/v1?rsz=filtered_cse&num=10&hl=en&source=gcsc&gss=.ai&cselibv=323d4b81541ddb5b&cx=000837339549978028235:ycech_9fn6e&q={encodedsearch}&safe=off&cse_tok=AJvRUv3g9NqQUYmQlWTzH9xdoRwz:1620337963675&sort=&exp=csqr,cc&oq={encodedsearch}&gs_l=partner-generic.12..0l3.57158.60259.1.70624.8.8.0.0.0.0.140.604.7j1.8.0.csems%2Cnrl%3D13...0.60246j2542254460j12j2...1.34.partner-generic..9.15.1042.n1wdxDnRleU&callback=google.search.cse.api9527"
    r = requests.get(url=searchurl)
    match = re.search('"url":\\s"([^"]+)",', r.text)
    if match:
        docurl = match.group(1)
        print(docurl)
    else:
        print("NO RESULT")

Concepts - Data loading
Concepts%20-%20Data%20loading
NO RESULT
Concepts - Model training
Concepts%20-%20Model%20training
https://docs.fast.ai/dev/abbr.html
Concepts - Learner lifecycle
Concepts%20-%20Learner%20lifecycle
https://docs.fast.ai/dev/abbr.html
Learner - Factory methods to create an instance
Learner%20-%20Factory%20methods%20to%20create%20an%20instance
https://docs.fast.ai/collab.html
Learner - Init & Attributes
Learner%20-%20Init%20%26%20Attributes
https://docs.fast.ai/learner.html
Learner - Training methods
Learner%20-%20Training%20methods
https://docs.fast.ai/learner.html
Learner - Inference methods
Learner%20-%20Inference%20methods
https://docs.fast.ai/learner.html
Diagnostics - How to debug
Diagnostics%20-%20How%20to%20debug
NO RESULT
Show - Inputs, targets and predictions
Show%20-%20Inputs%2C%20targets%20and%20predictions
https://docs.fast.ai/learner.html
Show - Images
Show%20-%20Images
https://docs.fast.ai/vision.core.html
Show - Text, points, boxes, tables
Show%20-%2